In [1]:
#presidency only comparison, 2016 data
#election date in november justifies 2016 data over 2015

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import os
import json

In [3]:
def BEA(method, **kwargs):
    '''
    methods: getdata, getdatasetlist, getparameterlist, getparametervalues, getparametervaluesfiltered
    params: datasetname, parametername, targetparameter, tablename, linecode, year, geoflips
    '''
    target = 'https://apps.bea.gov/api/data'
    payload = {'UserID': os.environ['BEA_API_KEY'], 'method':method}
    payload.update(kwargs)
    return requests.get(target, params=payload)

In [4]:
def clean_BEA(df,labelname):
    df=df[df['NoteRef'] != '(NA)']
    df=df[df['NoteRef'] != '*']
    df=df.drop(labels=['Code','CL_UNIT','NoteRef','UNIT_MULT','TimePeriod','GeoName'],axis=1)
    if ',' in df['DataValue'].iloc[0]:
        df['DataValue']=df['DataValue'].str.replace(',', '').astype(float)
    df=df.astype({'GeoFips':'float64','DataValue':'float64'})
    df=df.rename(columns={'DataValue':labelname})
    return df

In [5]:
#clean vote import
vote=pd.read_csv('files/countypres_2000-2016.csv')
vote.rename(columns = {'FIPS':'GeoFips'}, inplace = True)
vote=vote[vote['year']==2016]
vote=vote.dropna(axis=0,how='any')

In [6]:
vote.columns

Index(['year', 'state', 'state_po', 'county', 'GeoFips', 'office', 'candidate',
       'party', 'candidatevotes', 'totalvotes', 'version'],
      dtype='object')

In [6]:
y=vote[(vote['GeoFips']< 3000) & (vote['GeoFips']>2000)]

In [8]:
#df=pd.DataFrame.from_records(resp.json()['BEAAPI']['Results']['Data'])
#method 1

In [9]:
#df1=pd.read_json(json.dumps(resp.json()['BEAAPI']['Results']['Data']))
#method 2

In [10]:
#df=pd.DataFrame.from_dict(resp.json()['BEAAPI']['Results']['Data'])
#method 3

In [7]:
resp=requests.get('https://www.nrcs.usda.gov/wps/portal/nrcs/detail/?cid=nrcs143_013697')
#GeoFips data
df=pd.read_html(resp.text)[0]
df.drop(3232,inplace=True)
df.rename(columns={'FIPS':'GeoFips'}, inplace=True)
df=df.astype({'GeoFips':'float64'})
df=df.drop(labels=['Name','State'],axis=1)
merged=pd.merge(vote,df,how='inner',on='GeoFips')

In [8]:
x=df[(df['GeoFips']< 3000) & (df['GeoFips']>2000)]
x

,GeoFips
67,2013.0
68,2016.0
69,2020.0
70,2050.0
71,2060.0
72,2068.0
73,2070.0
74,2090.0
75,2100.0
76,2110.0


In [13]:
print(x.count())
print(y.count())

GeoFips    27
Name       27
State      27
dtype: int64
year              80
state             80
state_po          80
county            80
GeoFips           80
office            80
candidate         80
party             80
candidatevotes    80
totalvotes        80
version           80
dtype: int64


In [9]:
x.head() #gov county fips

,GeoFips
67,2013.0
68,2016.0
69,2020.0
70,2050.0
71,2060.0


In [10]:
y.head() #voting fips

,year,state,state_po,county,GeoFips,office,candidate,party,candidatevotes,totalvotes,version
41251,2016,Alaska,AK,District 1,2701.0,President,Hillary Clinton,democrat,2573.0,6638,20190722
41252,2016,Alaska,AK,District 1,2701.0,President,Donald Trump,republican,3180.0,6638,20190722
41254,2016,Alaska,AK,District 2,2702.0,President,Hillary Clinton,democrat,1585.0,5492,20190722
41255,2016,Alaska,AK,District 2,2702.0,President,Donald Trump,republican,3188.0,5492,20190722
41257,2016,Alaska,AK,District 3,2703.0,President,Hillary Clinton,democrat,1241.0,7613,20190722


In [13]:
y.tail()

,year,state,state_po,county,GeoFips,office,candidate,party,candidatevotes,totalvotes,version
41363,2016,Alaska,AK,District 38,2738.0,President,Donald Trump,republican,1143.0,5095,20190722
41365,2016,Alaska,AK,District 39,2739.0,President,Hillary Clinton,democrat,3142.0,5639,20190722
41366,2016,Alaska,AK,District 39,2739.0,President,Donald Trump,republican,1405.0,5639,20190722
41368,2016,Alaska,AK,District 40,2740.0,President,Hillary Clinton,democrat,2338.0,4610,20190722
41369,2016,Alaska,AK,District 40,2740.0,President,Donald Trump,republican,1377.0,4610,20190722


In [11]:
resp=BEA('getdata', datasetname='regional',geofips='County',tablename='CAEMP25N',year='2016',linecode=10)
#num jobs
df=pd.DataFrame.from_records(resp.json()['BEAAPI']['Results']['Data'])
df=clean_BEA(df,'num_jobs')
merged=pd.merge(merged,df,how='inner',on='GeoFips')

In [19]:
df.head()

,num_jobs,GeoFips
0,193368900.0,0.0
1,2620993.0,1000.0
2,17445.0,1001.0
3,107855.0,1003.0
4,11633.0,1005.0


In [12]:
df[(df['GeoFips']>2000) & (df['GeoFips']<3000)] #economic fips

,num_jobs,GeoFips
73,204969.0,2020.0
75,1458.0,2060.0
78,58427.0,2090.0
79,1721.0,2100.0
81,23430.0,2110.0
82,32216.0,2122.0
84,9482.0,2150.0
87,37814.0,2170.0
89,14420.0,2185.0
90,3159.0,2188.0


In [11]:
resp=BEA('getdata', datasetname='regional',geofips='County',tablename='CAINC30',year='2016', linecode=10)
#personal income total
df=pd.DataFrame.from_records(resp.json()['BEAAPI']['Results']['Data'])
df=clean_BEA(df,'total_personal_income_thous')
merged=pd.merge(merged,df,how='inner',on='GeoFips')

In [12]:
resp=BEA('getdata', datasetname='regional',geofips='County',tablename='CAINC45',year='2016',linecode=370)
#farm income thousands
df=pd.DataFrame.from_dict(resp.json()['BEAAPI']['Results']['Data'])
df=clean_BEA(df,'Farm_income_thousands')
merged=pd.merge(merged,df,how='inner',on='GeoFips')

In [13]:
resp=BEA('getdata', datasetname='regional',geofips='County',tablename='CAINC30',year='2016',linecode=110)
#income/capita
df=pd.DataFrame.from_dict(resp.json()['BEAAPI']['Results']['Data'])
df=clean_BEA(df,'Income/Capita')
merged=pd.merge(merged,df,how='inner',on='GeoFips')

In [14]:
resp=BEA('getdata', datasetname='regional',geofips='County',tablename='CAINC30',year='2016',linecode=100)
#population
df=pd.DataFrame.from_dict(resp.json()['BEAAPI']['Results']['Data'])
df=clean_BEA(df,'population')
merged=pd.merge(merged,df,how='inner',on='GeoFips')

In [15]:
resp=BEA('getdata', datasetname='regional',geofips='County',tablename='CAINC30',year='2016',linecode=290)
#average salary
df=pd.DataFrame.from_dict(resp.json()['BEAAPI']['Results']['Data'])
df=clean_BEA(df,'avg_sal')
merged=pd.merge(merged,df,how='inner',on='GeoFips')

In [16]:
df=pd.read_excel('files/laucnty16.xlsx', converters={'state_fips_Code':str, 'county_fips_code':str})
#unemployment file
df['GeoFips']=df['state_fips_Code']+df['county_fips_code']
df=df.astype({'GeoFips':'float64'})
df=df.drop(labels=['LAUS_Code','state_fips_Code','county_fips_code','Unnamed: 5', 'Year','County Name/State Abbreviation'], axis=1)
merged=pd.merge(merged,df,how='inner',on='GeoFips')

In [17]:
df=merged
df.head()

,year,state,state_po,county,GeoFips,office,candidate,party,candidatevotes,totalvotes,...,total_personal_income_thous,Farm_income_thousands,Income/Capita,population,avg_sal,labor_Force,Employed,Unemployed,Unemployment_Rate,share_vote
0,2016,Alabama,AL,Autauga,1001.0,President,Hillary Clinton,democrat,5936.0,24973,...,2183987.0,3073.0,39509.0,55278.0,36966.0,25966.0,24645.0,1321.0,5.1,0.237697
1,2016,Alabama,AL,Autauga,1001.0,President,Donald Trump,republican,18172.0,24973,...,2183987.0,3073.0,39509.0,55278.0,36966.0,25966.0,24645.0,1321.0,5.1,0.727666
2,2016,Alabama,AL,Baldwin,1003.0,President,Hillary Clinton,democrat,18458.0,95215,...,8923673.0,15316.0,43004.0,207509.0,36078.0,90670.0,85839.0,4831.0,5.3,0.193856
3,2016,Alabama,AL,Baldwin,1003.0,President,Donald Trump,republican,72883.0,95215,...,8923673.0,15316.0,43004.0,207509.0,36078.0,90670.0,85839.0,4831.0,5.3,0.765457
4,2016,Alabama,AL,Barbour,1005.0,President,Hillary Clinton,democrat,4871.0,10469,...,818333.0,28166.0,31750.0,25774.0,38418.0,8417.0,7717.0,700.0,8.3,0.465278


In [18]:
df.columns

Index(['year', 'state', 'state_po', 'county', 'GeoFips', 'office', 'candidate',
       'party', 'candidatevotes', 'totalvotes', 'version', 'num_jobs',
       'total_personal_income_thous', 'Farm_income_thousands', 'Income/Capita',
       'population', 'avg_sal', 'labor_Force', 'Employed', 'Unemployed',
       'Unemployment_Rate', 'share_vote'],
      dtype='object')

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6092 entries, 0 to 6091
Data columns (total 22 columns):
year                           6092 non-null int64
state                          6092 non-null object
state_po                       6092 non-null object
county                         6092 non-null object
GeoFips                        6092 non-null float64
office                         6092 non-null object
candidate                      6092 non-null object
party                          6092 non-null object
candidatevotes                 6092 non-null float64
totalvotes                     6092 non-null int64
version                        6092 non-null int64
num_jobs                       6092 non-null float64
total_personal_income_thous    6092 non-null float64
Farm_income_thousands          6092 non-null float64
Income/Capita                  6092 non-null float64
population                     6092 non-null float64
avg_sal                        6092 non-null float64
labo

In [33]:
df=df.drop(labels=['state_po','office','version'],axis=1)

In [34]:
df.to_pickle('./files/table.pkl')

In [ ]:
df.info()

In [21]:
df.iloc[0]

year                                      2016
state                                  Alabama
state_po                                    AL
county                                 Autauga
GeoFips                                   1001
office                               President
candidate                      Hillary Clinton
party                                 democrat
candidatevotes                            5936
totalvotes                               24973
version                               20190722
num_jobs                                 17445
total_personal_income_thous        2.18399e+06
Farm_income_thousands                     3073
Income/Capita                            39509
population                               55278
avg_sal                                  36966
labor_Force                              25966
Employed                                 24645
Unemployed                                1321
Unemployment_Rate                          5.1
share_vote   

In [22]:
vote.sort_values(by='GeoFips')

,year,state,state_po,county,GeoFips,office,candidate,party,candidatevotes,totalvotes,version
41050,2016,Alabama,AL,Autauga,1001.0,President,Hillary Clinton,democrat,5936.0,24973,20190722
41051,2016,Alabama,AL,Autauga,1001.0,President,Donald Trump,republican,18172.0,24973,20190722
41053,2016,Alabama,AL,Baldwin,1003.0,President,Hillary Clinton,democrat,18458.0,95215,20190722
41054,2016,Alabama,AL,Baldwin,1003.0,President,Donald Trump,republican,72883.0,95215,20190722
41056,2016,Alabama,AL,Barbour,1005.0,President,Hillary Clinton,democrat,4871.0,10469,20190722
41057,2016,Alabama,AL,Barbour,1005.0,President,Donald Trump,republican,5454.0,10469,20190722
41059,2016,Alabama,AL,Bibb,1007.0,President,Hillary Clinton,democrat,1874.0,8819,20190722
41060,2016,Alabama,AL,Bibb,1007.0,President,Donald Trump,republican,6738.0,8819,20190722
41062,2016,Alabama,AL,Blount,1009.0,President,Hillary Clinton,democrat,2156.0,25588,20190722
41063,2016,Alabama,AL,Blount,1009.0,President,Donald Trump,republican,22859.0,25588,20190722


In [23]:
#remarks:
#county is not divided by voting district. 
##Limitation cauced by FIPS (GeoFips) code for counties != voting district division
##Prevents analysis with extra weight to elected outcome
##Does not prevent economic profile to voting habit analysis
#Crude GeoFips to voting district may be done without damaging economic values


NameError: name 'f' is not defined

In [31]:
df.head()

,year,state,county,GeoFips,candidate,party,candidatevotes,totalvotes,version,num_jobs,total_personal_income_thous,Farm_income_thousands,Income/Capita,population,avg_sal,labor_Force,Employed,Unemployed,Unemployment_Rate,share_vote
0,2016,Alabama,Autauga,1001.0,Hillary Clinton,democrat,5936.0,24973,20190722,17445.0,2183987.0,3073.0,39509.0,55278.0,36966.0,25966.0,24645.0,1321.0,5.1,0.237697
1,2016,Alabama,Autauga,1001.0,Donald Trump,republican,18172.0,24973,20190722,17445.0,2183987.0,3073.0,39509.0,55278.0,36966.0,25966.0,24645.0,1321.0,5.1,0.727666
2,2016,Alabama,Baldwin,1003.0,Hillary Clinton,democrat,18458.0,95215,20190722,107855.0,8923673.0,15316.0,43004.0,207509.0,36078.0,90670.0,85839.0,4831.0,5.3,0.193856
3,2016,Alabama,Baldwin,1003.0,Donald Trump,republican,72883.0,95215,20190722,107855.0,8923673.0,15316.0,43004.0,207509.0,36078.0,90670.0,85839.0,4831.0,5.3,0.765457
4,2016,Alabama,Barbour,1005.0,Hillary Clinton,democrat,4871.0,10469,20190722,11633.0,818333.0,28166.0,31750.0,25774.0,38418.0,8417.0,7717.0,700.0,8.3,0.465278
